In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [26]:
import os
filename = []
list = os.listdir('/Users/HP/Documents/Cap/capstone/j_prepared_data')
for f in list:
    if "all" in f:
        filename.append(f)

In [27]:
filename

['1751_all.csv',
 '1801_all.csv',
 '1810_all.csv',
 '1818_all.csv',
 '1933_all.csv']

In [28]:

df = pd.DataFrame()
for f in filename:
    df_c = pd.read_csv("../j_prepared_data/"+f)
    df=df.append(df_c)

    

In [29]:
features = df[['mxamp', 'std', 'skew', 'kurtosis', 'hr',  'patpk', 'patft']]
target = df[['sbp', 'dbp']]

In [30]:
features

,mxamp,std,skew,kurtosis,hr,patpk,patft
0,2879.176501,751.728982,-0.229849,-0.933357,88.669951,0.2675,0.152500
1,2765.073311,719.716591,-0.357917,-0.762510,83.916084,1.1150,0.156667
2,2833.671027,778.133839,-0.388241,-0.945357,90.909091,1.5975,0.166250
3,2783.069263,743.363892,-0.791137,-0.341928,95.490716,2.1600,0.168750
4,2804.659607,790.950755,-0.638073,-0.764912,97.826087,1.4950,0.165000
...,...,...,...,...,...,...,...
82,1648.349920,454.933806,-0.313119,-0.754156,98.901099,1.5075,0.160000
83,1659.651143,448.757452,-0.422786,-0.687299,98.630137,1.5100,0.167500
84,1610.583462,436.938508,-0.561483,-0.606359,100.558659,2.0850,0.172500
85,1718.082386,451.968913,-0.571665,-0.592175,104.347826,1.4500,0.171250


In [31]:
def cindex(true_labels, predictions):
    n,h = 0,0
    
    for i in range(1,len(true_labels)):
        t = true_labels[i]
        p = predictions[i]
        for j in range(i+1,len(true_labels)):
            nt = true_labels[j]
            npr = predictions[j]
            if t != nt:
                n += 1
                if (p < npr and t < nt) or (p > npr and t > nt):
                    h += 1
                else :
                    h += 0.5
                    
    return h/n

In [32]:
cindex([1,1,1,4,5,8], [1,1,1,5,4,8])


0.9444444444444444

In [22]:
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
loo = LeaveOneOut()
X = features.values
y = target.values
sbp_er = []
dbp_er = []
true_label = []
pred_label = []

for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    
    model = make_pipeline(StandardScaler(),LinearRegression())
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae_sbp = mean_absolute_error(y_pred[:,0], y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred[:,1], y_test[:,1])
    
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    true_label.append(y_test)
    pred_label.append(y_pred)
true = [item for sublist in true_label for item in sublist]
pred = [item for sublist in pred_label for item in sublist]

true = np.array(true)
pred = np.array(pred)

cidx_sbp = cindex(true[:,0], pred[:,0])
cidx_dbp = cindex(true[:,1], pred[:,1])
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")

Average MAE SBP: 8.208646866302665 MAE DBP: 3.8478558600832264
C index SBP: 0.8406519726269409  C index DBP: 0.9019646449058653


In [33]:
from sklearn.ensemble import RandomForestRegressor

loo = LeaveOneOut()
X = features.values
y = target.values
sbp_er = []
dbp_er = []
true_label = []
pred_label = []

for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    
    forest = make_pipeline(StandardScaler(),RandomForestRegressor(n_estimators = 200))
    forest.fit(X_train, y_train)
    y_pred = forest.predict(X_test)
    
    mae_sbp = mean_absolute_error(y_pred[:,0], y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred[:,1], y_test[:,1])
    
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    true_label.append(y_test)
    pred_label.append(y_pred)
true = [item for sublist in true_label for item in sublist]
pred = [item for sublist in pred_label for item in sublist]

true = np.array(true)
pred = np.array(pred)

cidx_sbp = cindex(true[:,0], pred[:,0])
cidx_dbp = cindex(true[:,1], pred[:,1])
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")


Average MAE SBP: 4.085605783524897 MAE DBP: 2.5454678946360185
C index SBP: 0.9297687338363789  C index DBP: 0.9351326614233565


In [34]:
from sklearn.svm import SVR
svr = make_pipeline(StandardScaler(),SVR(kernel = 'rbf', C=10, epsilon = 0.01))
X = features.values
y = target.values
sbp_er = []
dbp_er = []
true_label_sbp = []
pred_label_sbp = []

true_label_dbp = []
pred_label_dbp= []

for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    

    svr.fit(X_train, y_train[:,0])
    y_pred_sbp = svr.predict(X_test)
    true_label_sbp.append(y_test[:,0])
    pred_label_sbp.append(y_pred_sbp)
    
    svr.fit(X_train, y_train[:,1])
    y_pred_dbp = svr.predict(X_test)
    true_label_dbp.append(y_test[:,1])
    pred_label_dbp.append(y_pred_dbp)
    
    mae_sbp = mean_absolute_error(y_pred_sbp, y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred_dbp, y_test[:,1])
    
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    
    
true_sbp = [item for sublist in true_label_sbp for item in sublist]
pred_sbp = [item for sublist in pred_label_sbp for item in sublist]

true_dbp = [item for sublist in true_label_dbp for item in sublist]
pred_dbp = [item for sublist in pred_label_dbp for item in sublist]

true_sbp = np.array(true_sbp)
pred_sbp = np.array(pred_sbp)

true_dbp = np.array(true_dbp)
pred_dbp = np.array(pred_dbp)

cidx_sbp = cindex(true_sbp, pred_sbp)
cidx_dbp = cindex(true_dbp, pred_dbp)
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")

Average MAE SBP: 5.202132262172269 MAE DBP: 2.8322151644157474
C index SBP: 0.907009291088856  C index DBP: 0.9275763348623365


In [35]:
pred_dbp

array([74.62310304, 75.03638157, 73.87538966, 73.66121109, 74.21025065,
       74.65336282, 75.22788049, 73.31451015, 72.41074262, 73.08678661,
       76.83221495, 75.78231682, 74.02171158, 73.87997615, 73.11476271,
       72.39585308, 71.79338616, 75.9619843 , 77.2990712 , 79.58501971,
       79.7359349 , 79.31359301, 79.79672441, 76.8532587 , 75.8888466 ,
       72.10285985, 70.25738937, 70.30152836, 73.84534077, 78.45252514,
       80.03021728, 75.72635517, 75.96457485, 70.37993829, 71.95064063,
       73.55717188, 77.47778767, 75.74792365, 75.30703131, 71.9363081 ,
       73.14281736, 69.5949622 , 71.85084464, 76.62287853, 78.61097026,
       77.09411181, 74.95866694, 72.64372235, 71.44196085, 69.39259053,
       70.45160187, 74.03549182, 75.64619096, 73.63501604, 70.44563564,
       68.7985868 , 69.76198953, 67.21945511, 70.02387734, 74.33996341,
       77.08642933, 77.74142151, 74.67050803, 72.1905139 , 71.39557573,
       67.28748835, 70.01348422, 72.10057461, 71.81181729, 69.85